In [1]:
import json

# load data
with open('./json/file_new_stopwords_token.json') as data_file:
    dataNormalPre = json.load(data_file)

In [21]:
count_v = CountVectorizer()
wo = count_v.fit_transform(["zbozi zboOi","zbozi"])
print (len (count_v.vocabulary_.items()))
# most common words
freqs = [(word, wo.getcol(idx).sum()) for word, idx in count_v.vocabulary_.items()]
#sort from largest to smallest
print (sorted (freqs, key = lambda x: -x[1])[:10])

2
[('zbozi', 2), ('zbooi', 1)]


In [2]:
# load right dataset
dataIteration = dataNormalPre

# for get ridding of non czech/english words
def filterNonWordsOut(dataIteration):
    dataIteration_preprocess = []

    words_order = dataIteration
    only_english_words = []

    # to keep only english words
    with open("./wordlist_cz.txt") as word_file:
        only_english_words = set(word.strip().lower() for word in word_file)

    for line in words_order:
        english_line = ""
        for w in line.split():
            if w.strip().lower() in english_words_file:
                english_line += str(w.strip().lower()) + " "
            else: 
                pass
        dataIteration_preprocess.append(english_line)    
    return dataIteration_preprocess

In [22]:
#divide data into two parts -> plus/minus

words = []

labels = []


#every review
for x in range(0,len(dataIteration)):

    
    for plus in  dataIteration[x].get("plus"):
        words.append(plus)
        labels.append(1)
        

    for minus in dataIteration[x].get("minus"):
        words.append(minus)
        labels.append(0)
        


In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(words, labels, test_size=0.2, random_state=0)

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

words_train_count = count_vect.fit_transform(words)

In [27]:
len (count_vect.vocabulary_.items())

18599

In [25]:
# most common words
freqs = [(word, words_train_count.getcol(idx).sum()) for word, idx in count_vect.vocabulary_.items()]
#sort from largest to smallest
print (sorted (freqs, key = lambda x: -x[1])[:10])

[('cena', 4147), ('rychlý', 2731), ('dobrý', 2095), ('zboží', 1682), ('žádný', 1367), ('vědět', 1123), ('kvalita', 953), ('zatím', 921), ('kvalitní', 859), ('komunikace', 830)]


In [55]:
from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer = TfidfTransformer(use_idf=False).fit(words_train_count)
words_train_tf = tf_transformer.transform(words_train_count)

tfidf_transformer = TfidfTransformer()
words_train_tfidf = tfidf_transformer.fit_transform(words_train_tf)

In [56]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(words_train_tfidf, y_train)

In [57]:
docs_new = X_test
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

print("Score " + str(clf.score(X_new_tfidf, y_test)))

from sklearn.metrics import f1_score
predicted = clf.predict(X_new_tfidf)
# F1 = 2 * (precision * recall) / (precision + recall) 
print (f1_score(y_pred=predicted, y_true=y_test, average="binary")) ## binary is most suitable for us -> 0/1
print (f1_score(y_pred=predicted, y_true=y_test, average="macro"))
print (f1_score(y_pred=predicted, y_true=y_test, average="micro"))
print (f1_score(y_pred=predicted, y_true=y_test, average="weighted"))
print (f1_score(y_pred=predicted, y_true=y_test, average=None))

Score 0.618511918022
0.713700576778
0.571100246674
0.618511918022
0.571004946832
[ 0.42849992  0.71370058]


In [58]:
from sklearn.metrics import roc_auc_score

predicted_proba = clf.predict_proba(X_new_tfidf)
# Compute Area Under the Curve (AUC) from prediction scores
print ("ROC :" + str(roc_auc_score(y_test, predicted_proba[:, 1])))

ROC :0.847172156058


In [6]:
from __future__ import print_function

from pprint import pprint
from time import time
import logging

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Sample pipeline for text feature extraction and evaluation
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer='word', max_df=0.5, stop_words=None, max_features=None)),
    ('tfidf', TfidfTransformer(use_idf=True, norm="l1")),
    ('clf', SGDClassifier()),
    #    ('clf', MultinomialNB()),
])

with open('./czech_stop_words.txt') as stop_words:
    stop_word_list = list([f.strip() for f in stop_words])

# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
# parameters = {
#     'vect__max_df': (0.5, 0.75, 1.0),
#     'vect__stop_words' : (stop_word_list ,'english', None),
#     'vect__max_features': (None, 5000, 10000, 50000),
#     'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
#     'vect__analyzer': ('word', 'char', 'char_wb') ,
#     'tfidf__use_idf': (True, False),
#     'tfidf__norm': ('l1', 'l2'),
#     #'clf__penalty': ('l2', 'elasticnet'),
#     #'clf__n_iter': (10, 50, 80),
# }

parameters = {
    'clf__penalty': ('l2', 'elasticnet'),
    'clf__n_iter': (10, 50, 80),
}

# find the best parameters for both the feature extraction and the
# classifier
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(words, labels)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__n_iter': (10, 50, 80), 'clf__penalty': ('l2', 'elasticnet')}
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   11.1s finished


done in 11.939s

Best score: 0.749
Best parameters set:
	clf__n_iter: 10
	clf__penalty: 'l2'


In [9]:
from sklearn.model_selection import train_test_split
# testing chi square features selection 
with open('./czech_stop_words.txt') as stop_words:
    stop_word_list = list([f.strip() for f in stop_words])

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

feats = []
scores = []
f1s = []
rocs = []

for featureSelected in range(1000, 22000, 1000):
    X_train, X_test, y_train, y_test = train_test_split(words, labels, test_size=0.2, random_state=0)

    X_train = count_vect.fit_transform(X_train)
    #chi2 = SelectKBest(chi2, k=featureSelected)
    #chi2 = SelectKBest(mutual_info_classif, k=featureSelected)
    chi2 = SelectKBest(f_classif, k=featureSelected)

        
    
    words_train_count = chi2.fit_transform(X_train, y_train)
    
    print ("Len :" + str(words_train_count.shape))
    
    X_test = count_vect.transform(X_test)
    X_test = chi2.transform(X_test)

    tf_transformer = TfidfTransformer(use_idf=True).fit(words_train_count)
    words_train_tf = tf_transformer.transform(words_train_count)

    tfidf_transformer = TfidfTransformer()
    words_train_tfidf = tfidf_transformer.fit_transform(words_train_tf)

    clf = MultinomialNB().fit(words_train_tfidf, y_train)

    score = clf.score(X_test, y_test)
    predicted = clf.predict(X_test)
    predicted_proba = clf.predict_proba(X_test)
    
    
    f1 = f1_score(y_pred=predicted, y_true=y_test, average="binary")
    roc = roc_auc_score(y_test, predicted_proba[:, 1])

    feats.append(featureSelected)
    scores.append(score)
    f1s.append(f1)
    rocs.append(roc)
    
    print("Feat : " + str(featureSelected) + "\tScore :" + str(score) + "\t" + " f1 : " + str(f1) + "\t ROC : " + str(roc))
    
for x in range(len(feats)):
    print(feats[x])

print()
for x in range(len(scores)):
    print(scores[x])

print()
for x in range(len(f1s)):
    print(f1s[x])
print()
    
for x in range(len(rocs)):
    print(rocs[x])

Len :(35908, 1000)
Feat : 1000	Score :0.60793049677	 f1 : 0.70802919708	 ROC : 0.832492767645
Len :(35908, 2000)
Feat : 2000	Score :0.614947649811	 f1 : 0.711844627824	 ROC : 0.840357643787
Len :(35908, 3000)
Feat : 3000	Score :0.616952550679	 f1 : 0.714012474012	 ROC : 0.844476448744
Len :(35908, 4000)
Feat : 4000	Score :0.616507017153	 f1 : 0.713155044572	 ROC : 0.843869162158
Len :(35908, 5000)
Feat : 5000	Score :0.617843617732	 f1 : 0.715104209914	 ROC : 0.846947999694
Len :(35908, 6000)
Feat : 6000	Score :0.617063934061	 f1 : 0.715585704831	 ROC : 0.847944048944
Len :(35908, 7000)
Feat : 7000	Score :0.616507017153	 f1 : 0.715853759181	 ROC : 0.848032232663
Len :(35908, 8000)
Feat : 8000	Score :0.615504566719	 f1 : 0.715791206982	 ROC : 0.848037666606
Len :(35908, 9000)
Feat : 9000	Score :0.615281799955	 f1 : 0.715907221582	 ROC : 0.848049502181
Len :(35908, 10000)
Feat : 10000	Score :0.615393183337	 f1 : 0.715779076467	 ROC : 0.848059228692
Len :(35908, 11000)
Feat : 11000	Score :

ValueError: k should be >=0, <= n_features; got 17000.Use k='all' to return all features.